### Constructing YouTube Links

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/musiccaps-public.csv')

In [4]:
df.head(1)

,ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval
0,-0Gj8-vB1q4,30,40,"/m/0140xf,/m/02cjck,/m/04rlf","['low quality', 'sustained strings melody', 's...",The low quality recording features a ballad so...,4,False,True


In [9]:
for row in df.iterrows():
    print(row[1])
    break

ytid                                                              -0Gj8-vB1q4
start_s                                                                    30
end_s                                                                      40
audioset_positive_labels                         /m/0140xf,/m/02cjck,/m/04rlf
aspect_list                 ['low quality', 'sustained strings melody', 's...
caption                     The low quality recording features a ballad so...
author_id                                                                   4
is_balanced_subset                                                      False
is_audioset_eval                                                         True
Name: 0, dtype: object


In [10]:
link_base = "https://www.youtube.com/watch?v="

In [27]:
for row in df.iterrows():
    final_link = link_base + row[1][0]
    start_time = row[1][1]
    end_time = row[1][2]
    loc = f'./audio/audio_{row[0]}.mp3'
    !yt-dlp --extract-audio --audio-format mp3 --postprocessor-args "-ss {start_time} -to {end_time}" -o "{loc}" {final_link}
    break

[youtube] Extracting URL: https://www.youtube.com/watch?v=-0Gj8-vB1q4
[youtube] -0Gj8-vB1q4: Downloading webpage
[youtube] -0Gj8-vB1q4: Downloading tv client config
[youtube] -0Gj8-vB1q4: Downloading player 73381ccc-main
[youtube] -0Gj8-vB1q4: Downloading tv player API JSON
[youtube] -0Gj8-vB1q4: Downloading ios player API JSON
[youtube] -0Gj8-vB1q4: Downloading m3u8 information
[info] -0Gj8-vB1q4: Downloading 1 format(s): 251
[download] Destination: audio\audio_0.webm

[download]   0.0% of    3.79MiB at  997.69KiB/s ETA 00:03
[download]   0.1% of    3.79MiB at    1.46MiB/s ETA 00:02
[download]   0.2% of    3.79MiB at    3.42MiB/s ETA 00:01
[download]   0.4% of    3.79MiB at    7.32MiB/s ETA 00:00
[download]   0.8% of    3.79MiB at    4.12MiB/s ETA 00:00
[download]   1.6% of    3.79MiB at    4.84MiB/s ETA 00:00
[download]   3.3% of    3.79MiB at    6.69MiB/s ETA 00:00
[download]   6.6% of    3.79MiB at    8.94MiB/s ETA 00:00
[download]  13.2% of    3.79MiB at   12.42MiB/s ETA 00:00
[do

ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


In [ ]:
# Define variables
start_time = "00:01:30"
end_time = "00:02:00"
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
output_location = "/path/to/output/folder/%(title)s.%(ext)s"  # Specify your desired output folder

# Use the variables in the shell command
!yt-dlp --extract-audio --audio-format mp3 --postprocessor-args "-ss {start_time} -to {end_time}" -o "{output_location}" {video_url}


In [ ]:
!yt-dlp --extract-audio --audio-format mp3 --postprocessor-args "-ss 30 -to 40" -o "./audio/test" https://www.youtube.com/watch?v=-0Gj8-vB1q4

[youtube] Extracting URL: https://www.youtube.com/watch?v=-0Gj8-vB1q4
[youtube] -0Gj8-vB1q4: Downloading webpage
[youtube] -0Gj8-vB1q4: Downloading tv client config
[youtube] -0Gj8-vB1q4: Downloading player 73381ccc-main
[youtube] -0Gj8-vB1q4: Downloading tv player API JSON
[youtube] -0Gj8-vB1q4: Downloading ios player API JSON
[youtube] -0Gj8-vB1q4: Downloading m3u8 information
[info] -0Gj8-vB1q4: Downloading 1 format(s): 251
[download] Destination: audio.test

[download]   0.0% of    3.79MiB at  999.83KiB/s ETA 00:03
[download]   0.1% of    3.79MiB at    1.46MiB/s ETA 00:02
[download]   0.2% of    3.79MiB at    3.42MiB/s ETA 00:01
[download]   0.4% of    3.79MiB at    5.85MiB/s ETA 00:00
[download]   0.8% of    3.79MiB at    8.63MiB/s ETA 00:00
[download]   1.6% of    3.79MiB at    8.19MiB/s ETA 00:00
[download]   3.3% of    3.79MiB at    9.53MiB/s ETA 00:00
[download]   6.6% of    3.79MiB at   13.09MiB/s ETA 00:00
[download]  13.2% of    3.79MiB at   15.04MiB/s ETA 00:00
[download] 

ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location
